In [1]:
import numpy as np

In [2]:
def initialize_parameters(neurons_per_layer):
    ## initialize weights and biases of ANN
    
    L = len(neurons_per_layer) - 1
    parameters = dict()
    
    for i in range(1, L + 1):
        
        n_neurons = neurons_per_layer[i]
        n_neurons_prev = neurons_per_layer[i - 1]

        parameters[f'W{i}'] = np.sqrt(1 / n_neurons) * np.random.randn(
                            n_neurons, n_neurons_prev)
        parameters[f'b{i}'] = np.sqrt(1 / n_neurons) * np.random.randn(n_neurons)
        
    return parameters

In [3]:
np.random.seed(42)

In [4]:
neurons_per_layer = [10, 5, 5, 3]
initialize_parameters(neurons_per_layer)

{'W1': array([[ 0.22213732, -0.06183368,  0.28965512,  0.68111966, -0.10471657,
         -0.10470923,  0.70624544,  0.34320724, -0.20995533,  0.24264023],
        [-0.20724669, -0.20828068,  0.10820882, -0.85564494, -0.77140671,
         -0.25146263, -0.45295185,  0.14053568, -0.40608071, -0.63160142],
        [ 0.65545806, -0.10097023,  0.03019953, -0.63716676, -0.24345536,
          0.04960609, -0.51473998,  0.16801726, -0.26861379, -0.13044941],
        [-0.26909138,  0.82836399, -0.00603614, -0.47302271,  0.36785327,
         -0.54597788,  0.09340664, -0.87639112, -0.59398286,  0.08803902],
        [ 0.33025229,  0.07663823, -0.05171948, -0.13465767, -0.66121514,
         -0.32192412, -0.20600392,  0.47275943,  0.15367077, -0.78845553]]),
 'b1': array([ 0.14493476, -0.17221403, -0.30272872,  0.27354995,  0.461077  ]),
 'W2': array([[ 0.41648113, -0.37530949, -0.13828398,  0.14814551,  0.43627704],
        [-0.21429323, -0.08302922, -0.49476804, -0.53495987,  0.36337259],
        [ 

In [5]:
import pandas as pd

In [6]:
## load dataset
dataset = pd.read_csv('./data/MNIST_preprocessed.csv', sep=",", header=None).values

## set input, target variables
X = dataset[:, 0:784]
Y = dataset[:, 784:]

X_train, X_test = X[0:250,], X[250:,]
Y_train, Y_test = Y[0:250,], Y[250:,]

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
def feed_forward(x, parameters):
    
    cache = {'a0' : x}
    L = len(parameters) // 2
    
    for l in range(1, L + 1):
        a_prev = cache[f'a{l - 1}']
        W = parameters[f'W{l}']
        b = parameters[f'b{l}']
        
        cache[f'z{l}'] = W @ a_prev + b
        cache[f'a{l}'] = sigmoid(cache[f'z{l}'])
    
    return cache[f'a{L}'], cache

In [9]:
## run feed forward with 'MNIST_preprocessed.csv' data
np.random.seed(42)

neurons_per_layer = [784, 128, 64, 10]
parameters = initialize_parameters(neurons_per_layer)
feed_forward(X_train[0], parameters)[0]

array([0.39847348, 0.63079802, 0.79832892, 0.93056447, 0.67941405,
       0.67578969, 0.05318435, 0.37468117, 0.12677036, 0.6419338 ])

In [10]:
## practice back propagation

In [11]:
def d_sigmoid(x):
    """시그모이드 미분 함수"""
    return (np.exp(-x))/((np.exp(-x)+1)**2)

In [18]:
def back_prop(prediction, y, cache, parameters):
    """역전파 함수"""
    gradients = {}
    L = len(cache) // 2
    da = (prediction - y) / y.shape[0]
    
    for layer in range(L, 0, -1):
        # 역전파 행렬 연산을 사용해서 각 요소에 대한 편미분 계산
        # 코드를 쓰세요
        db = d_sigmoid(cache[f'z{layer}']) * da
        dW = np.outer(db, cache[f'a{layer - 1}'])
        da = parameters[f'W{layer}'].T @ db
        
        
        # 계산한 편미분 값들을 저장
        gradients['dW' + str(layer)] = dW
        gradients['db' + str(layer)] = db
    
    # 계산한 편미분 값들 리턴
    return gradients

In [13]:
def compute_accuracy(x_val, y_val, parameters):
    """예측 값들의 성능을 계산하는 함수"""
    
    predictions = []
    
    for x, y in zip(x_val, y_val):
        output, _ = feed_forward(x, parameters)
        pred = np.argmax(output)
        predictions.append(pred == np.argmax(y))
    
    return np.mean(predictions)

In [14]:
def compute_loss(x_val, y_val, parameters):
    """모델의 손실을 계산하는 함수"""
    
    loss = 0
    for x, y in zip(x_val, y_val):
        output, _ = feed_forward(x, parameters)
        loss += np.mean((y - output)**2) / 2
    
    return loss / len(x_val)

In [15]:
def update(parameters, gradients, alpha, m):
    """계산된 경사로 가중치와 편향 값을 업데이트하는 함수"""
    
    L = len(parameters) // 2
    
    for layer in range(1, L+1):
        parameters[f'W{layer}'] -= alpha * gradients[f'dW{layer}'] / m
        parameters[f'b{layer}'] -= alpha * gradients[f'db{layer}'] / m
    
    return parameters

In [16]:
def train_nn(X_train, Y_train, X_test, Y_test, neurons_per_layer, epoch, alpha):
    """신경망 모델을 학습시키는 함수"""
    
    parameters = initialize_parameters(neurons_per_layer)
    loss_list = []
    m = X_train.shape[0]
    
    for i in range(epoch):
        parameters_copy = parameters.copy()
        
        for x, y in zip(X_train, Y_train):
            prediction, cache = feed_forward(x, parameters)
            gradients = back_prop(prediction, y, cache, parameters)
            parameters_copy = update(parameters_copy, gradients, alpha, m)
        
        parameters = parameters_copy
        loss_list.append(compute_loss(X_train, Y_train, parameters))
        accuracy = round(compute_accuracy(X_test, Y_test, parameters), 2)
        print(f'{i+1}번째 경사 하강, 테스트 셋에서 성능 : {accuracy}')
    
    return loss_list, parameters

In [19]:
np.random.seed(42)

neurons_per_layer = [784, 128, 64, 10]
parameters = initialize_parameters(neurons_per_layer)
loss_list, parameters = train_nn(X_train, Y_train, X_test, Y_test, neurons_per_layer, 25, 300)


1번째 경사 하강, 테스트 셋에서 성능 : 0.16
2번째 경사 하강, 테스트 셋에서 성능 : 0.18
3번째 경사 하강, 테스트 셋에서 성능 : 0.36
4번째 경사 하강, 테스트 셋에서 성능 : 0.54
5번째 경사 하강, 테스트 셋에서 성능 : 0.64
6번째 경사 하강, 테스트 셋에서 성능 : 0.64
7번째 경사 하강, 테스트 셋에서 성능 : 0.64
8번째 경사 하강, 테스트 셋에서 성능 : 0.68
9번째 경사 하강, 테스트 셋에서 성능 : 0.68
10번째 경사 하강, 테스트 셋에서 성능 : 0.7
11번째 경사 하강, 테스트 셋에서 성능 : 0.7
12번째 경사 하강, 테스트 셋에서 성능 : 0.7
13번째 경사 하강, 테스트 셋에서 성능 : 0.7
14번째 경사 하강, 테스트 셋에서 성능 : 0.7
15번째 경사 하강, 테스트 셋에서 성능 : 0.76
16번째 경사 하강, 테스트 셋에서 성능 : 0.76
17번째 경사 하강, 테스트 셋에서 성능 : 0.78
18번째 경사 하강, 테스트 셋에서 성능 : 0.8
19번째 경사 하강, 테스트 셋에서 성능 : 0.8
20번째 경사 하강, 테스트 셋에서 성능 : 0.8
21번째 경사 하강, 테스트 셋에서 성능 : 0.8
22번째 경사 하강, 테스트 셋에서 성능 : 0.8
23번째 경사 하강, 테스트 셋에서 성능 : 0.8
24번째 경사 하강, 테스트 셋에서 성능 : 0.8
25번째 경사 하강, 테스트 셋에서 성능 : 0.8
